In [6]:
# Install the mysql-connector-python and faker packages
%pip install mysql-connector-python faker
import mysql.connector
import random
from faker import Faker
from datetime import datetime, timedelta
fake = Faker()

# MySQL Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "crop_management"
}

conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

# Create crops table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS crops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    crop_name VARCHAR(255) NOT NULL,
    planting_date DATE,
    harvest_date DATE,
    growth_stage VARCHAR(255),
    pest_control_measures TEXT,
    yield_prediction INT
)
""")
conn.commit()

# List of sample crop names
crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]

# List of possible growth stages
growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]

# List of sample pest control measures
pest_control_measures_list = [
    "Use of organic pesticides",
    "Crop rotation",
    "Neem oil application",
    "Biological pest control",
    "Chemical pesticides",
    "Regular field monitoring",
]

# Function to generate random data for crops
def generate_data():
    crop_name = random.choice(crop_names)
    planting_date = fake.date_between(start_date="-2y", end_date="today")  # Planting in last 2 years
    harvest_date = planting_date + timedelta(days=random.randint(60, 180))  # Harvest after 2-6 months
    growth_stage = random.choice(growth_stages)
    pest_control = random.choice(pest_control_measures_list)
    yield_prediction = random.randint(500, 5000)  # Yield in kg
    
    return (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction)

# Batch insert records in chunks
batch_size = 100000  # Insert 10,000 at a time for efficiency
total_records = 2000000

for i in range(0, total_records, batch_size):
    data_batch = [generate_data() for _ in range(batch_size)]
    
    # Execute batch insert
    cursor.executemany("""
        INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, data_batch)
    conn.commit()
    
    print(f"{i + batch_size} records inserted...")

print("✅ Data insertion completed successfully!")

# Close the database connection
cursor.close()
conn.close()

Note: you may need to restart the kernel to use updated packages.
100000 records inserted...
200000 records inserted...
300000 records inserted...
400000 records inserted...
500000 records inserted...
600000 records inserted...
700000 records inserted...
800000 records inserted...
900000 records inserted...
1000000 records inserted...
1100000 records inserted...
1200000 records inserted...
1300000 records inserted...
1400000 records inserted...
1500000 records inserted...
1600000 records inserted...
1700000 records inserted...
1800000 records inserted...
1900000 records inserted...
2000000 records inserted...
✅ Data insertion completed successfully!


In [7]:
def fetch_table_data(table_name):
    # The connect() constructor creates a connection to the MySQL server and returns a MySQLConnection object.
    cnx = mysql.connector.connect(
        host='localhost',
        database='crop_management',
        user='root',
        password='root'
    )

    cursor = cnx.cursor()
    cursor.execute('select * from ' + table_name)

    header = [row[0] for row in cursor.description]

    rows = cursor.fetchall()

    # Closing connection
    cnx.close()

    return header, rows
def export(table_name):
    header, rows = fetch_table_data(table_name)

    # Create csv file
    f = open(table_name + '.csv', 'w')

    # Write header
    f.write(','.join(header) + '\n')

    for row in rows:
        f.write(','.join(str(r) for r in row) + '\n')

    f.close()
    print(str(len(rows)) + ' rows written successfully to ' + f.name)


# Tables to be exported
export('crops')

2000000 rows written successfully to crops.csv


In [8]:
import pandas as pd
df = pd.read_csv("crops.csv")
df

,id,crop_name,planting_date,harvest_date,growth_stage,pest_control_measures,yield_prediction
0,1,Sugarcane,2023-10-07,2024-03-05,Vegetative,Crop rotation,780
1,2,Tomato,2023-10-08,2024-04-03,Flowering,Chemical pesticides,2769
2,3,Barley,2024-12-14,2025-06-05,Flowering,Neem oil application,1454
3,4,Tomato,2024-07-27,2024-12-31,Flowering,Use of organic pesticides,2094
4,5,Corn,2024-07-13,2024-10-29,Seedling,Crop rotation,942
...,...,...,...,...,...,...,...
1999995,1999996,Sugarcane,2024-03-08,2024-06-14,Seedling,Biological pest control,1128
1999996,1999997,Lettuce,2025-02-13,2025-06-05,Maturity,Use of organic pesticides,880
1999997,1999998,Potato,2024-11-17,2025-01-20,Fruiting,Regular field monitoring,4490
1999998,1999999,Potato,2023-11-21,2024-04-30,Seedling,Biological pest control,884
